In [ ]:


import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon




In [ ]:


# Load the shapefile
shapefile_path = "C:\\Data set\\UoA\\ne_110m_populated_places\\ne_110m_populated_places.shp"
gdf = gpd.read_file(shapefile_path)




In [ ]:


# Plot the data
gdf.plot()
plt.show()




In [ ]:


print(gdf.head())




In [ ]:


# Access the geometry (points, lines, or polygons)
geometry = gdf.geometry

# Access the attribute data
attributes = gdf.drop(columns='geometry')




In [ ]:


from shapely.geometry import Point




In [ ]:


bbox_data = {'geometry': [Point(-100, 30), Point(-90, 40)]}
bbox_gdf = gpd.GeoDataFrame(bbox_data, crs=gdf.crs)




In [ ]:


# Perform spatial join to find points within the bounding box
points_within_bbox = gpd.sjoin(gdf, bbox_gdf, predicate='intersects')




In [ ]:


output_shapefile_path = 'C:\Data set\geodata\output_shapefile.shp'
gdf.to_file(output_shapefile_path)




In [ ]:


print(points_within_bbox.head())
print(points_within_bbox.info())




In [ ]:


print(gdf.is_valid)  # Check validity of geometries in gdf
print(bbox_gdf.is_valid)  # Check validity of geometries in bbox_gdf




In [ ]:


print(gdf)




In [ ]:


print(bbox_gdf)




In [ ]:


import matplotlib.pyplot as plt

# Plot the points in gdf and the bounding box in bbox_gdf on the same plot
ax = gdf.plot(markersize=5, color='blue', label='Points in gdf')
bbox_gdf.plot(ax=ax, markersize=50, color='red', label='Bounding Box')
plt.legend()
plt.show()


# To find the cities that are non-Eurocentric locations the code is given below



In [ ]:


# Load the GeoDataFrame from the provided files (CPG, DBF, PRJ, SHP, SHX)
file_path = r'C:\Data set\UoA\ne_110m_populated_places' # Replace this with the directory where your files are located
gdf = gpd.read_file(file_path)




In [ ]:


# Define a bounding box for Europe as a single polygon
europe_bbox_poly = Polygon([(-24, 34), (-24, 71), (40, 71), (40, 34)])
europe_bbox_gdf = gpd.GeoDataFrame({'geometry': [europe_bbox_poly]}, crs='EPSG:4326')




In [ ]:


# Perform spatial join to find cities outside Europe
non_eurocentric_cities = gpd.sjoin(gdf, europe_bbox_gdf, how='left', predicate='intersects')
non_eurocentric_cities = non_eurocentric_cities[non_eurocentric_cities['index_right'].isnull()]




In [ ]:


# Print the column names in the non_eurocentric_cities GeoDataFrame
print(non_eurocentric_cities.columns)


# # USING NLP WE ARE DETECTING THE CITY IN A GIVEN SENTENCE



In [ ]:


import requests
import spacy
from fuzzywuzzy import fuzz
from opencage.geocoder import OpenCageGeocode
from langdetect import detect
from googletrans import Translator




In [ ]:


# Load spaCy's English language model
nlp = spacy.load('en_core_web_sm')

def extract_location_entities(sentence, lang='en'):
    if lang != 'en':
        translator = Translator()
        sentence = translator.translate(sentence, src=lang, dest='en').text

    doc = nlp(sentence)
    locations = []

    for ent in doc.ents:
        if ent.label_ == 'GPE':  # GPE: Geo-Political Entity (location)
            locations.append((ent.text, ent.start_char, ent.end_char))

    return locations

def get_location_info(location, api_key):
    try:
        geocoder = OpenCageGeocode(api_key)
        results = geocoder.geocode(location)

        if results and len(results):
            place = results[0]
            return {
                'name': place['formatted'],
                'country': place['components']['country'],
                'latitude': place['geometry']['lat'],
                'longitude': place['geometry']['lng']
            }
        else:
            return None

    except Exception as e:
        print(f"Error: {e}")
        return None




In [ ]:


# Example usage
api_key = "08e5bf766e5842b880d7e140ce85d343"  # Replace with your actual OpenCageData API key
user_input = input("Enter the text (in any language) containing city or country names: ")

# Detect the language of the input text
lang = detect(user_input)

locations = extract_location_entities(user_input, lang)

if locations:
    print("Location Entities:")
    for location in locations:
        info = get_location_info(location[0], api_key)
        if info:
            print(f"Original Name: {location[0]}")
            print(f"English Name: {info['name']}")
            print(f"Country: {info['country']}")
            print(f"Latitude: {info['latitude']}, Longitude: {info['longitude']}")
            print("=" * 30)
        else:
            print(f"Location '{location[0]}' not found.")
else:
    print("No location entities found in the sentence.")


